In [1]:
import sqlite3

DB_NAME = '../databases/dataset.db'
BUILDINGS_TABLE = 'buildings'

conn = None
try:
    conn = sqlite3.connect(DB_NAME)
    cursor = conn.cursor()
    
    print(f"Успешное подключение к базе данных: {DB_NAME}\n")
    print("--- Анализ наличия координат по статусу здания (is_fake) ---")

    # ----------------------------------------------------------------------
    # SQL-запрос для подсчета всех четырех комбинаций
    # ----------------------------------------------------------------------
    analysis_query = f"""
    SELECT 
        is_fake,
        SUM(CASE WHEN coordinates IS NOT NULL THEN 1 ELSE 0 END) AS has_coords_count,
        SUM(CASE WHEN coordinates IS NULL THEN 1 ELSE 0 END) AS no_coords_count,
        COUNT(*) AS total_count
    FROM {BUILDINGS_TABLE}
    GROUP BY is_fake
    ORDER BY is_fake DESC;
    """
    
    cursor.execute(analysis_query)
    results = cursor.fetchall()
    
    # ----------------------------------------------------------------------
    # Обработка и вывод результатов
    # ----------------------------------------------------------------------
    
    # Заголовки
    print(f"\n| {'Статус (is_fake)':<18} | {'С координатами (NOT NULL)':<25} | {'Без координат (NULL)':<22} | {'Всего в группе':<15} |")
    print("|" + "-"*19 + "|" + "-"*27 + "|" + "-"*24 + "|" + "-"*17 + "|")
    
    total_buildings = 0
    
    for is_fake, has_coords, no_coords, total in results:
        status = "ФЕЙКОВОЕ (1)" if is_fake == 1 else "РЕАЛЬНОЕ (0)"
        total_buildings += total
        
        print(f"| {status:<18} | {has_coords:<25} | {no_coords:<22} | {total:<15} |")

    print("|" + "-"*19 + "|" + "-"*27 + "|" + "-"*24 + "|" + "-"*17 + "|")
    print(f"\nВсего записей в таблице buildings: {total_buildings}")


except sqlite3.Error as e:
    print(f"❌ Произошла ошибка SQLite: {e}")
finally:
    if conn:
        conn.close()
        print("\nСоединение с БД закрыто.")

Успешное подключение к базе данных: ../databases/dataset.db

--- Анализ наличия координат по статусу здания (is_fake) ---

| Статус (is_fake)   | С координатами (NOT NULL) | Без координат (NULL)   | Всего в группе  |
|-------------------|---------------------------|------------------------|-----------------|
| ФЕЙКОВОЕ (1)       | 4402                      | 7678                   | 12080           |
| РЕАЛЬНОЕ (0)       | 46119                     | 0                      | 46119           |
|-------------------|---------------------------|------------------------|-----------------|

Всего записей в таблице buildings: 58199

Соединение с БД закрыто.
